In [1]:
import requests
import ast
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from google.colab import files
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_csv('/content/drive/MyDrive/player_metadata.csv')

In [6]:
df

,player_id,name,nameColor,trophies,highestTrophies,highestPowerPlayPoints,expLevel,expPoints,isQualifiedFromChampionshipChallenge,3vs3Victories,soloVictories,duoVictories,bestRoboRumbleTime,bestTimeAsBigBrawler,brawlers,icon_id,club_tag,club_name
0,#8Y9QGR0U,nat ♡,0xffcb5aff,41103,42113,544.0,247,313184,False,18746,1332,663,1,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000186,#2RYVPRP2J,brawl stars
1,#99QU9PU2,tylo ♡,0xff1ba5f5,51754,55004,1204.0,411,858011,True,64497,900,1132,9,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000088,#2GCQ9UU8P,NA Friends
2,#GG9YJJY,flowerm8,0xffa8e132,21897,35635,1013.0,310,491272,False,37032,1217,600,0,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 9...",28000044,NaN,NaN
3,#VJUQ0Y,DW | Chino🐰,0xffcb5aff,57372,61076,1380.0,412,862979,True,63416,1348,780,5,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000041,#2022UQ08V,Havoc Echo
4,#PJGGV2Q80,DW | charlz🖤,0xffcb5aff,59129,61757,1362.0,359,654132,True,48834,558,2838,13,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000003,#2RCQ9CGQQ,STMN Esports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22733,#829VV99,OS | Mateo,0xffcb5aff,34161,41017,1289.0,305,473536,False,30472,894,1104,5,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000016,NaN,NaN
22734,#8UP2PCCJQ,ally<3,0xffa8e132,45451,46553,779.0,348,614767,True,42284,335,1565,11,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000018,#2GCQ9UU8P,NA Friends
22735,#8L89CJL,DarkUwU,0xffffffff,27912,37568,957.0,307,481867,False,29546,616,1737,5,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 9...",28000170,NaN,NaN
22736,#2JGUC2C0,匚尺乇卂爪卄卂乙乇,0xffa2e3fe,38360,52570,821.0,295,444029,False,28550,927,4255,8,0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1...",28000078,NaN,NaN


In [7]:
Xf = df['expLevel']

In [8]:
Ytemp = df['brawlers']
Ytemp

,brawlers
0,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."
1,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."
2,"[{'id': 16000000, 'name': 'SHELLY', 'power': 9..."
3,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."
4,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."
...,...
22733,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."
22734,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."
22735,"[{'id': 16000000, 'name': 'SHELLY', 'power': 9..."
22736,"[{'id': 16000000, 'name': 'SHELLY', 'power': 1..."


In [9]:
Yresult = []
for i in range(len(Xf)):
    try:
        variable = Ytemp[i]

        start = variable.find("[")
        end = variable.rfind("]") + 1

        brawlers_data = variable[start:end]


        brawlers_list = ast.literal_eval(brawlers_data)

        power_11_count = sum(1 for brawler in brawlers_list if brawler.get('power') == 11)
        power_11_score = power_11_count / 65

        Yresult.append(power_11_score)

    except (SyntaxError, ValueError, Exception) as e:
        print(f"Error processing index {i}: {e}")
        Yresult.append(0)

print(Yresult)

[0.9846153846153847, 0.6153846153846154, 0.0, 1.0, 0.9846153846153847, 0.3384615384615385, 0.6615384615384615, 0.2923076923076923, 0.07692307692307693, 0.9846153846153847, 0.3230769230769231, 0.16923076923076924, 0.6615384615384615, 0.2923076923076923, 0.07692307692307693, 0.9846153846153847, 0.3230769230769231, 0.16923076923076924, 0.6615384615384615, 0.2923076923076923, 0.07692307692307693, 0.9846153846153847, 0.3230769230769231, 0.16923076923076924, 0.9846153846153847, 0.9230769230769231, 0.9230769230769231, 0.0, 0.046153846153846156, 0.0, 0.9230769230769231, 0.9846153846153847, 0.9230769230769231, 0.0, 0.046153846153846156, 0.0, 0.9230769230769231, 0.9846153846153847, 0.9230769230769231, 0.0, 0.046153846153846156, 0.0, 0.9230769230769231, 0.9230769230769231, 0.9846153846153847, 0.0, 0.046153846153846156, 0.0, 0.9230769230769231, 0.9846153846153847, 0.9230769230769231, 0.0, 0.046153846153846156, 0.0, 0.9846153846153847, 0.9230769230769231, 0.9230769230769231, 0.046153846153846156, 0

In [10]:
Yresults = pd.DataFrame(Yresult, columns = ['Power 11 Score'])
Yresults

,Power 11 Score
0,0.984615
1,0.615385
2,0.000000
3,1.000000
4,0.984615
...,...
22733,0.092308
22734,0.815385
22735,0.046154
22736,0.276923


In [11]:
X = [Xf, Yresults]

In [12]:
y = df['trophies']

In [13]:
X = pd.concat([Xf, Yresults], axis=1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

In [16]:
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f'Mean Absolute Percentage Error: {mape}%')

Mean Absolute Percentage Error: 7.373661448705085%


In [17]:
new_data = pd.DataFrame([[198, 1]], columns=['expLevel', 'Power 11 Score'])
prediction = tree.predict(new_data)

print(f'Predicted number of trophies: {prediction[0]}')

Predicted number of trophies: 42294.0


In [19]:
joblib.dump(tree, 'model.pkl')

['model.pkl']

In [20]:
files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>